In [1]:
using Random
using LinearAlgebra
rng = MersenneTwister(2018)
import LinearAlgebra.dot
import LinearAlgebra.norm
import LinearAlgebra.triu
import LinearAlgebra.diag
using Printf

In [2]:
A = [2 -4 5 -1;4 2 1 -3;-3 1 2 -6;3 -6 5 2]
# A = [2 -1 4; 3 4 -2; 1 4 2; -4 -1 3]
A = float(A)
A0 = copy(A)
B0 = copy(A)
C0 = copy(A)

4×4 Matrix{Float64}:
  2.0  -4.0  5.0  -1.0
  4.0   2.0  1.0  -3.0
 -3.0   1.0  2.0  -6.0
  3.0  -6.0  5.0   2.0

In [3]:
function house(x)
    """Computes the Householder transformation for input vector x"""
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


house (generic function with 1 method)

In [4]:
#Householder transformation
    m = size(A,1)
    n = size(A,2)
    vA = zeros(n)
    E = Matrix{Float64}(I, m, m)
    kend = (m > n ? n : m-1)
    #display(kend)
   Q = copy(E)
    for k=1:kend
        beta, v = house(A[k:end,k])
    G = Matrix{Float64}(I, m-k+1, m-k+1) - beta*v*v'

    P = copy(E)
    P = float(P)
    P[k:end, k:end] = G
 
    Q = Q*P
    
    #v = v/v(1)
        for j=k:n
            # vA = beta * v^T * A
            vA[j] = 0.0
            for i=k:m
                vA[j] += v[i-k+1] * A[i,j]
            end
            vA[j] *= beta
        end
        # A - beta v (v^T A)
        for j=k:n, i=k:m
            A[i,j] -= v[i-k+1] * vA[j]
        end
    v = v/v[1]
        A[k+1:end,k] = v[2:end]
        # Saving v in the lower triangular part of A.
        # This was not done here but one can always
        # divide v by v[1] such that v[1] = 1 is always true.
        # In that case, v[1] does not need to be stored.
    end
    R = triu(A)
    display("R")
    display(R)
    display("Q")
    display(Q)

"R"

4×4 Matrix{Float64}:
 -6.16441   3.40665  -3.73109  -1.62221
  0.0      -6.73756   4.94088   2.14821
  0.0       0.0      -4.08248   6.53197
  0.0       0.0       0.0      -0.294884

"Q"

4×4 Matrix{Float64}:
 -0.324443   0.429642   -0.408248     -0.73721
 -0.648886  -0.624933   -0.408248      0.147442
  0.486664   0.0976458  -0.816497      0.294884
 -0.486664   0.644462    1.45035e-17   0.589768

In [5]:
H = R[:,[1,3,4]]  #deleting second column of R

4×3 Matrix{Float64}:
 -6.16441  -3.73109  -1.62221
  0.0       4.94088   2.14821
  0.0      -4.08248   6.53197
  0.0       0.0      -0.294884

In [6]:
function givens(a, b)
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return (c, s)
end

givens (generic function with 1 method)

In [7]:
# function givens_QR(A)
    n = size(H,1)
    m = size(H,2)
    G = Matrix(1.0I, n, n)
    for k=1:m
        c, s = givens(H[k,k], H[k+1,k])
        T = Matrix(1.0I, n, n) # Identity matrix of Float64 type
        T[k,k] = T[k+1, k+1] = c; T[k,k+1]= -s
        T[k+1,k] = s; 
        G = T*G
#         display("T")
#         display(T)
        # Apply the Givens rotation to row k and k+1
        for j=k:m
            H[k,j], H[k+1,j] =( c * H[k,j] - s * H[k+1,j], s * H[k,j] + c * H[k+1,j] )
        end
    end
        display("G")
        display(G)
        display("H")
        display(H)

"G"

4×4 Matrix{Float64}:
 1.0  0.0         0.0         0.0
 0.0  0.770894   -0.636964    0.0
 0.0  0.63629     0.770078   -0.0459996
 0.0  0.0293001   0.0354608   0.998941

"H"

4×3 Matrix{Float64}:
 -6.16441  -3.73109      -1.62221
  0.0       6.40929      -2.50459
  0.0      -4.44089e-16   6.41057
  0.0       0.0          -5.55112e-17

In [1]:
G #Givens rotator

LoadError: UndefVarError: `G` not defined

In [9]:
R1 = H #Upper triangular matrix R1

4×3 Matrix{Float64}:
 -6.16441  -3.73109      -1.62221
  0.0       6.40929      -2.50459
  0.0      -4.44089e-16   6.41057
  0.0       0.0          -5.55112e-17

In [10]:
Q1 = Q*G' #orthogonal matrix Q1

4×4 Matrix{Float64}:
 -0.324443   0.591247  -0.0070951  -0.738318
 -0.648886  -0.221718  -0.718804    0.114498
  0.486664   0.595353  -0.580199    0.268479
 -0.486664   0.496812   0.382936    0.608026

In [11]:
A1 = Q1*R1 #second column deleted

4×3 Matrix{Float64}:
  2.0  5.0  -1.0
  4.0  1.0  -3.0
 -3.0  2.0  -6.0
  3.0  5.0   2.0

In [12]:
A0 #given matrix

4×4 Matrix{Float64}:
  2.0  -4.0  5.0  -1.0
  4.0   2.0  1.0  -3.0
 -3.0   1.0  2.0  -6.0
  3.0  -6.0  5.0   2.0